In [1]:
!pip install map_boxes
!pip install tqdm
!pip install pycocotools

In [2]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

Couldn't import fast version of function compute_overlap, will use slow one. Check cython intallation


In [3]:
"""
    GT_JSON: valid set json file path
    현재 예시에서는 train.json을 임의로 valid set으로 사용
    PRED_CSV: 위의 valid set을 inference한 submission file
    Tip: validation json을 만들어서 validation set의 model의 결과를 추출하고 이의 mAP를 계산해보아요.
"""
GT_JSON = "../../dataset/train.json"
PRED_CSV = "../../sample_submission/train_sample.csv"
LABEL_NAME = [
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
]


# load ground truth
with open(GT_JSON, "r") as outfile:
    test_anno = json.load(outfile)

# load prediction
pred_df = pd.read_csv(PRED_CSV)

In [4]:
"""
[
    [file_name label_index confidence_score x_min x_max y_min y_max], 
    [file_name label_index confidence_score x_min x_max y_min y_max],
    ,,,
    [file_name label_index confidence_score x_min x_max y_min y_max]
]

아래의 mean_average_precision_for_boxes 함수에 들어갈 양식, new_pred 양식
"""

new_pred = []

file_names = pred_df["image_id"].values.tolist()
bboxes = pred_df["PredictionString"].values.tolist()

"""
create new_pred
"""

for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f"{file_names[i]} empty box")

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(" "))

    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception("error", "invalid box count")
    for box in boxes:
        new_pred.append(
            [
                file_name,
                box[0],
                box[1],
                float(box[2]),
                float(box[4]),
                float(box[3]),
                float(box[5]),
            ]
        )

4883it [00:01, 4741.87it/s]


In [5]:
"""
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2, confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]

아래의 mean_average_precision_for_boxes 함수에 들어갈 양식, gt 양식
"""

gt = []

"""
create gt
"""

coco = COCO(GT_JSON)

"""
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
"""
"""
image id 별로 GT 값 추가 (여기서는 test.json을 사용하였기 때문에 GT가 없습니다.)
"""
for image_id in coco.getImgIds():

    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info["id"])
    annotation_info_list = coco.loadAnns(annotation_id)

    file_name = image_info["file_name"]

    for annotation in annotation_info_list:
        gt.append(
            [
                file_name,
                annotation["category_id"],
                float(annotation["bbox"][0]),
                float(annotation["bbox"][0]) + float(annotation["bbox"][2]),
                float(annotation["bbox"][1]),
                (float(annotation["bbox"][1]) + float(annotation["bbox"][3])),
            ]
        )

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [6]:
"""
calculate mAP
"""


mean_ap, average_precisions = mean_average_precision_for_boxes(
    gt, new_pred, iou_threshold=0.5
)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.264624 |    3966
1                              | 0.522890 |    6352
2                              | 0.441265 |     897
3                              | 0.411529 |     936
4                              | 0.604017 |     982
5                              | 0.397688 |    2943
6                              | 0.490678 |    1263
7                              | 0.722987 |    5178
8                              | 0.599785 |     159
9                              | 0.531735 |     468
mAP: 0.498720
0.49871974228579613
